In [ ]:
import os, sys 
import pydicom as dicom
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import glob
import shutil
import tensorflow as tf
import cv2
import pandas as pd


def importOwnLib():
    if '/home/miruware/aProjects/lib' not in sys.path:
        sys.path.append('/home/miruware/aProjects/lib')
        print("lib path is successfully appended.")
    else:
        print("lib path is already exists.")

importOwnLib()
import sonyalib as sonya

import importlib
importlib.reload(sonya)

In [2]:
def get_windowing(data):
    if 'RescaleSlope' in data:
        dicom_fields = [data[('0028','1050')].value, #window center
                        data[('0028','1051')].value, #window width
                        data[('0028','1052')].value, #intercept
                        data[('0028','1053')].value] #slope
    else:
        dicom_fields = [data[('0028','1050')].value, #window center
                        data[('0028','1051')].value, #window width
                        0, #intercept
                        1] #slope
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]


def window_image(img, window_center,window_width, intercept, slope):
    img = (img * slope + intercept)
    img_min = window_center - window_width//2
    img_max = window_center + window_width//2 
    
    img[img <= img_min] = img_min
    img[img > img_max] = img_max
    
    return img 


def get_first_of_dicom_field_as_int(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == dicom.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)


def dcm2img(dcm, mode=None):
    window_center , window_width, intercept, slope = get_windowing(dcm)
    print(window_center , window_width, intercept, slope)
    image_windowed = window_image(dcm.pixel_array, window_center, window_width, intercept, slope)
    if mode == 'RGB' or 'rgb':
        img_gray = Image.fromarray(image_windowed).convert('L')
        img_result = Image.merge("RGB", (img_gray, img_gray, img_gray))
    else:
        img_result = Image.fromarray(image_windowed).convert('L')

    return img_result

def search_ref_dcm(nUID, nPath):
    # nUID : referenced dcm UID of annotation object
    # nPath : path to case which currently working on
    # return: PIL Image (pixel_array from dcm)
    for series_nonQ in [list_series for list_series in os.listdir(nPath) if list_series.find('Q') < 0 and not list_series.startswith('.')]:  # iterate series except Q series
        for nDcm in [list_dcm for list_dcm in os.listdir(os.path.join(PATH_SERIES, series_nonQ)) if list_dcm.endswith('.dcm')]:  # check extension
            with dicom.read_file(os.path.join(PATH_SERIES, series_nonQ, nDcm)) as dcm:
                if dcm.SOPInstanceUID == matching_uid:  # searching target dcm with Q
                    img_ans = dcm2img(dcm, mode='RGB')  # create new images for answer
                    return img_ans, series_nonQ, nDcm
    
                
def search_ref_dcm_name(nUID, nPath, dPath):
    # nUID : referenced dcm UID of annotation object
    # nPath : path to case which currently working on
    # return: PIL Image (pixel_array from dcm)
    for series_nonQ in [list_series for list_series in os.listdir(nPath) if list_series.find('Q') < 0 and not list_series.startswith('.')]:  # iterate series except Q series
        for nDcm in [list_dcm for list_dcm in os.listdir(os.path.join(nPath, series_nonQ)) if list_dcm.endswith('.dcm')]:  # check extension
            abspath = os.path.join(nPath, series_nonQ, nDcm)
            with dicom.read_file(abspath) as dcm:
                if dcm.SOPInstanceUID == matching_uid:  # searching target dcm with Q
                    return nDcm, abspath     
                else:
                    len_path = len(nPath.split('/'))
                    src = '/'.join(nPath.split('/')[0:len_path-1])
                    
                    if not os.path.exists(src):
                        print("There is no matching dicom file.")
                        shutil.copytree(src, dPath)  # move case to noQ directory
                    else:
                        return False, False
    
def getUID(path): ## path with Case Number
    for currentdir, dirs, files in os.walk(path):
        for file in files:
            if "Q" in file:
#                 print(file)
                return dicom.read_file(os.path.join(currentdir, file)).SOPInstanceUID

In [6]:
PATH_BASE = '/mnt/8TBDisk/KTL2020_Effusion/KTL2020_Effusion/'
PATH_noQ = '/mnt/8TBDisk/KTL2020_Effusion/noQ'
sonya.createFolder(PATH_noQ)
PATH_debug = '/mnt/8TBDisk/KTL2020_Effusion/debug_require'
sonya.createFolder(PATH_debug)
PATH_RESULT = '/mnt/8TBDisk/KTL2020_Effusion/result'
sonya.createFolder(PATH_RESULT)
PATH_PNG = '/mnt/8TBDisk/KTL2020_Effusion/png 변환/'


label_width = 10
arrow_width = 10

test_count = 0
for nPatient in [list_patient for list_patient in os.listdir(PATH_BASE) if not list_patient.startswith('.')]:

    PATH_PATIENT = os.path.join(PATH_BASE, nPatient)
    UID_check = []
#     print(PATH_PATIENT, "+++++")
    for nCase in [list_case for list_case in os.listdir(PATH_PATIENT) if not list_case.startswith('.')]:  # search Q file series
        PATH_CASE = os.path.join(PATH_PATIENT, nCase)
        # 여기서 중복체크가 이루어 져야 함.
        
        #####=============================================############
        if len([list_SeriesQ for list_SeriesQ in os.listdir(PATH_CASE) if list_SeriesQ.find('Q') >=0]) == 0: # check no Q cases
#             PATH_SERIES = os.path.join(PATH_CASE, )
#             move to non-Q directory
            shutil.move(PATH_CASE, os.path.join(PATH_noQ, nPatient, nCase))  # move case to noQ directory
            if len(os.listdir(PATH_PATIENT)) == 0: # Q파일이 없는 케이스를 옮겼는데 해당 환자폴더에 더이상 케이스가 남지 않은 경우, 해당 환자폴더 삭제
                os.rmdir(PATH_PATIENT)
        else: # chaeck overlapping cases
            currentUID = getUID(PATH_CASE)
            if currentUID in UID_check:
                print("there is overlapping cases." + os.path.join(nPatient, nCase))

            else: # extract Q file
                UID_check.append(currentUID) # SAVE CURRENT UID IN LOG
                nSeriesQ = [list_SeriesQ for list_SeriesQ in sorted(os.listdir(PATH_CASE)) if list_SeriesQ.find('Q') >=0][-1] # get latest q file

                PATH_SERIES = os.path.join(PATH_CASE, nSeriesQ)
                for nQ in [list_Q for list_Q in os.listdir(PATH_SERIES) if list_Q.endswith('.dcm')]:  # iterate all Q(dcm) file
                    dcm_q = dicom.read_file(os.path.join(PATH_SERIES, nQ))
                    if "GraphicAnnotationSequence" in dcm_q:
                        for nAnnotation in range(len(dcm_q.GraphicAnnotationSequence)):
                            # ================================= Searching Referenced SOP Instance UID to match dicom file ===============================
                            matching_uid = dcm_q.GraphicAnnotationSequence[nAnnotation].ReferencedImageSequence[0].ReferencedSOPInstanceUID
#                                 if matching_uid not in UID_check:
#                                     UID_check.append(matching_uid)
#                                 else:
#                                     print("there is overlapping cases." + os.path.join(nPatient, nCase))
#                             print(PATH_CASE)
                            
                            name_dcm, path_dcm = search_ref_dcm_name(matching_uid, PATH_CASE, os.path.join(PATH_debug, nPatient)) # matched dcm image name per annotation sequence
#                               print(name_dcm)
#                                 name_png = name_dcm.replace('.dcm','.png')
                            if name_dcm:
                                img_ans = Image.open(os.path.join(PATH_PNG, name_dcm.replace('.dcm','.png'))) # RGB 이미지
                                img_black = Image.new(mode="RGB", size = img_ans.size, color=(0,0,0)) # Black 이미지

                                # ========= Label Drawing =========
                                draw_rgb = ImageDraw.Draw(img_ans)
                                draw_black = ImageDraw.Draw(img_black) # draw RED label

                                if "GraphicObjectSequence" in dcm_q.GraphicAnnotationSequence[nAnnotation]:
                                    for nA_Object in range(len(dcm_q.GraphicAnnotationSequence[nAnnotation].GraphicObjectSequence)):
                                        #print(nDcm + "\t" + dcm_q.GraphicAnnotationSequence[nAnnotation].GraphicObjectSequence[nA_Object].GraphicType)
                                        obj_shape = dcm_q.GraphicAnnotationSequence[nAnnotation].GraphicObjectSequence[nA_Object].NumberOfGraphicPoints
                                        obj_graphic_data = dcm_q.GraphicAnnotationSequence[nAnnotation].GraphicObjectSequence[nA_Object].GraphicData

                                        if obj_shape == 2:  #
                                            print("Object shape is Circle - pass")
#                                             draw_rgb.ellipse(obj_graphic_data, width=label_width, outline="red")
#                                             draw_black.ellipse(obj_graphic_data, width=label_width, outline="red")
#                                            # img = Image.alpha_composite(answer_img, answer_back)
                                        elif obj_shape == 4:  # Ellipse/Oval 타원
                                            print("Object shape is Ellipse")
                                            #  ellipse_data = []
                                            obj_pt_X = sorted(obj_graphic_data[::1])
                                            obj_pt_Y = sorted(obj_graphic_data[1::2])
                                            ellipse_data = [min(obj_pt_X), min(obj_pt_Y), max(obj_pt_X), max(obj_pt_Y)]
                                            draw_rgb.ellipse(list(map(int, ellipse_data)), width=label_width, outline="red")
                                            draw_black.ellipse(list(map(int, ellipse_data)), width=label_width, outline="red")

                                            # img = Image.alpha_composite(answer_img, answer_back)
                                        elif obj_shape == 5 or 1:  # Polyline/Rect 자유곡선
                                            print("Object shape is Polyline")
                                            polyline_data = []
                                            for i in range(obj_shape):
                                                polyline_data.append(tuple(obj_graphic_data[i*2:i*2+2]))
                                            draw_rgb.line(polyline_data, width=label_width, fill='red', joint='curve')
                                            draw_black.line(polyline_data, width=label_width, fill='red', joint='curve')

                                          #  img = Image.alpha_composite(answer_img, answer_back)
                                        else:  # Polygon 다각형
                                            print("Object shape is Polygon")
                                            print(obj_shape, "\t", obj_graphic_data)
                                            draw_rgb.polygon(obj_graphic_data, outline='red')
                                            draw_black.polygon(obj_graphic_data, outline='red')


                                draw_rgb_name = os.path.join(PATH_RESULT, nQ.split('.')[0] + "_RGB.png") # rgb 저장
                                draw_black_name = os.path.join(PATH_RESULT, nQ.split('.')[0] + "_black.png") # black 저장
                                shutil.copy(path_dcm, PATH_RESULT) # 원본영상 복사
                                print(draw_rgb_name)

                                img_ans.save(draw_rgb_name)
                                img_black.save(draw_black_name)

    #####==========================================###########
         
            
            
            
#             

Object shape is Circle - pass
Object shape is Circle - pass
Object shape is Polyline
/mnt/8TBDisk/KTL2020_Effusion/result/100516_Q-20210119114629_14490883_0001_RGB.png
Object shape is Circle - pass
Object shape is Polyline
/mnt/8TBDisk/KTL2020_Effusion/result/100315_Q-20210121095557_14490282_0001_RGB.png
Object shape is Polyline
/mnt/8TBDisk/KTL2020_Effusion/result/100316_Q-20210121095616_14490283_0001_RGB.png
Object shape is Circle - pass
Object shape is Circle - pass
Object shape is Circle - pass
Object shape is Polyline
/mnt/8TBDisk/KTL2020_Effusion/result/101009_Q-20210120101315_14492426_0001_RGB.png
Object shape is Circle - pass
Object shape is Circle - pass
Object shape is Polyline
/mnt/8TBDisk/KTL2020_Effusion/result/101068_Q-20210122114920_14492635_0001_RGB.png
Object shape is Polyline
/mnt/8TBDisk/KTL2020_Effusion/result/102942_Q-20210121102824_14501729_0001_RGB.png
Object shape is Polyline
/mnt/8TBDisk/KTL2020_Effusion/result/101218_Q-20210122100857_14493402_0001_RGB.png
Obje